In [1]:
!nvidia-smi

Tue May 28 17:26:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  |   00000000:03:00.0 Off |                  N/A |
| 21%   33C    P8             10W /  250W |       1MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
torch.cuda.empty_cache()  # Free up unutilized memory

In [3]:
from src.segmentation.framework_handlers import ultralytics_handler
ultralytics_handler.validate_version_and_gpu()

Ultralytics YOLOv8.2.24 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
Setup complete ✅ (12 CPUs, 125.2 GB RAM, 15.0/336.4 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.
WE can process the data by two options:
- Single SegmentsAI dataset
- Two SegmentsAI datasets, one for train and the other for test

In [4]:
from src.annotation_handling import annotation_handler

Option 1: prepare data using a single dataset by splitting to train and test

In [ ]:
# prepare data from a single dataset to fine tune yolo v5
train_annotations_folder_name = "etaylor_cannabis_patches_test"
train_dataset_version = "v0.1"
saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO"
yaml_file_path = annotation_handler.convert_coco_to_yolo_format_from_single_dataset(train_annotations_folder_name, train_dataset_version, saving_yaml_path, train_percentage=0.8)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json:  53%|█████▎    | 47/89 [00:00<00:00, 465.70it/s]

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json: 100%|██████████| 89/89 [00:00<00:00, 479.47it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo


The folder /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo has been deleted.


Option 2: prepare dataset by using two different segments datasets, one for train and second for eval

In [ ]:
# prepare data from a train and test datasets to fine tune yolo v5
train_annotations_folder_name = "etaylor_cannabis_patches_train_26-04-2024_15-44-44"
train_dataset_version = "v0.1"
test_annotations_folder_name = "etaylor_cannabis_patches_test_26-04-2024_15-44-44"
test_dataset_version = "v0.1"
saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/yaml_files"
yaml_file_path = annotation_handler.convert_coco_to_yolo_format_from_train_test_datasets(train_annotations_folder_name, 
                                                                                        train_dataset_version, 
                                                                                        test_annotations_folder_name,
                                                                                        test_dataset_version,
                                                                                        saving_yaml_path, 
                                                                                        )

### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [4]:
from ultralytics import settings
import config

# Update a setting
yolov10_runs_dir = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/yolov10"
settings.update({'runs_dir': yolov10_runs_dir, 'weights_dir': config.ULTRALYTICS_WEIGHTS_DIR, 'datasets_dir': config.ULTRALYTICS_DATASETS_DIR})

# View all settings
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/datasets', 'weights_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/weights', 'runs_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/yolov10', 'uuid': '0def00e56c3a400afc7ba4172404e1311f32262b0c5348e767a82883f2e1b76d', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


In [5]:
import os

# in case dataset already converted from coco to yolo format use manual path
yaml_file_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/yaml_files/yolo_dataset_config.yaml"

# currently available models for train and hparams fine tune
ultralytics_checkpoint_paths = "/home/etaylor/code_projects/thesis/checkpoints/ultralytics"

available_models_checkpoints = {
    "yolov5": "yolov5xu.pt",
    "yolov8": "yolov8x.pt",
    "yolov9": "yolov9c.pt",
    "rt_detr": "rtdetr-x.pt",
    "yolov10": "yolov10n.pt",
}
model_checkpoint_path = os.path.join(ultralytics_checkpoint_paths, available_models_checkpoints["yolov10"])

Here we can select between using the yolo models or the RT_DETR models

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov10x.pt")

If using the RT DETR models use the code below

In [ ]:
# from ultralytics import RTDETR

# model_checkpoint_path = os.path.join(ultralytics_checkpoint_paths, available_models_checkpoints["rt_detr"])
# model = RTDETR(model_checkpoint_path)

In [7]:
# regular train of the model
results = model.train(data=yaml_file_path, epochs=1000, imgsz=512, batch=8)

New https://pypi.org/project/ultralytics/8.2.12 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.9 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48647MiB)
engine/trainer: task=detect, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/ultralytics/yolov8x.pt, data=/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/yaml_files/yolo_dataset_config.yaml, epochs=1000, time=None, patience=100, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, so

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/annotations/yolo_split/labels.cache... 369 images, 395 backgrounds, 0 corrupt: 100%|██████████| 764/764 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/home/etaylor/.conda/envs/yolov9/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/yolo_split/labels.cache... 106 images, 108 backgrounds, 0 corrupt: 100%|██████████| 214/214 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/yolov9/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoade

Plotting labels to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

   136/1000      5.29G     0.7012       0.44     0.8713         82        512: 100%|██████████| 96/96 [00:07<00:00, 13.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:00<00:00, 14.12it/s]

                   all        214       1525      0.501      0.544      0.513      0.363
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 36, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



136 epochs completed in 0.401 hours.
Optimizer stripped from /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train/weights/last.pt, 136.7MB
Optimizer stripped from /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train/weights/best.pt, 136.7MB

Validating /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.9 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48647MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  6.17it/s]


                   all        214       1525      0.543      0.628      0.593      0.409
                 clear        214        549      0.544      0.649      0.603      0.429
                cloudy        214        862      0.539      0.833      0.735      0.541
                 amber        214        114      0.547      0.404      0.441      0.256
Speed: 0.1ms preprocess, 2.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train


In [10]:
valid_results = model.val()
print(valid_results)

Ultralytics YOLOv8.1.46 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11017MiB)
Model summary (fused): 268 layers, 68127420 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/yolo_split/labels.cache... 106 images, 108 backgrounds, 0 corrupt: 100%|██████████| 214/214 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/yolov9/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:05<00:00,  2.78it/s]


                   all        214       1525      0.548      0.626      0.599      0.418
              trichome        214        549      0.423      0.778      0.607      0.439
                 clear        214        862      0.661      0.741      0.746      0.546
                cloudy        214        114      0.559       0.36      0.445      0.269
Speed: 0.1ms preprocess, 17.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs/detect/train102
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f96967511b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/sise/home/etaylor/runs/segment/train22/val_batch0_pred.jpg")

### Perform Infrence with the model

In [8]:
import cv2

image = cv2.imread("/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train/v0.1/IMG_2198_p12.png")

# Perform inference
results = model.predict(image)


0: 640x640 (no detections), 19.5ms
Speed: 12.6ms preprocess, 19.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
